In [1]:
# ===============================
# Notebook 03 — Financial Impact & Blocked Benefit Estimation
# Project Antyodaya (UIDAI)
# ===============================

import pandas as pd
import numpy as np
import os

pd.set_option("display.max_columns", None)


In [2]:
input_file = "../data/processed/rar_district_scores.csv"
df = pd.read_csv(input_file)

print("Input shape:", df.shape)
df.head()


Input shape: (1058, 7)


,state,district,RaR_score,ghost_cohort_flag,child_updates_5_17,age_17_updates,pincode_count
0,Andaman & Nicobar Islands,Andamans,0.780905,1,231,1250,9.0
1,Andaman & Nicobar Islands,Nicobars,0.799969,1,1,1,1.0
2,Andaman & Nicobar Islands,South Andaman,0.787953,1,32,139,6.0
3,Andaman and Nicobar Islands,Nicobar,0.786768,1,824,691,4.0
4,Andaman and Nicobar Islands,North And Middle Andaman,0.774435,1,2830,2310,7.0


In [3]:
df["ghost_cohort_estimated"] = (
    df["RaR_score"] * df["child_updates_5_17"]
).round().astype(int)

df[["district", "RaR_score", "ghost_cohort_estimated"]].head()


,district,RaR_score,ghost_cohort_estimated
0,Andamans,0.780905,180
1,Nicobars,0.799969,1
2,South Andaman,0.787953,25
3,Nicobar,0.786768,648
4,North And Middle Andaman,0.774435,2192


In [4]:
AVERAGE_BENEFIT_PER_CHILD = 12000  # INR


In [5]:
df["blocked_value_rs"] = (
    df["ghost_cohort_estimated"] * AVERAGE_BENEFIT_PER_CHILD
)

df["blocked_value_cr"] = df["blocked_value_rs"] / 1e7  # Convert to Crores

df[["district", "blocked_value_cr"]].head()


,district,blocked_value_cr
0,Andamans,0.2160
1,Nicobars,0.0012
2,South Andaman,0.0300
3,Nicobar,0.7776
4,North And Middle Andaman,2.6304


In [6]:
df["priority_index"] = (
    0.6 * df["RaR_score"] +
    0.4 * (df["blocked_value_cr"] / df["blocked_value_cr"].max())
)


In [7]:
df["priority_category"] = pd.cut(
    df["priority_index"],
    bins=[0, 0.4, 0.7, 1.0],
    labels=["Monitor", "Action Soon", "Immediate Action"]
)

df["priority_category"].value_counts()


priority_category
Action Soon         1027
Monitor               25
Immediate Action       6
Name: count, dtype: int64

In [8]:
output_file = "../data/processed/rar_financial_impact.csv"
df.to_csv(output_file, index=False)

print("Saved Financial Impact CSV to:", output_file)
print("Files in processed folder:", os.listdir("../data/processed"))


Saved Financial Impact CSV to: ../data/processed/rar_financial_impact.csv
Files in processed folder: ['combined_uidai_data.csv', 'rar_district_scores.csv', 'rar_financial_impact.csv']
